In [1]:
# OpenCV - 이미지 읽기, 쓰기 및 표시하기 (1)
import numpy as np
import cv2

def handle_image():
    imgfile = 'images/sample.png'
    img = cv2.imread(imgfile, cv2.IMREAD_COLOR) # cv2.imread로 이미지를 불러옴 (이미지 경로, 이미지불러올 때의 설정 Flag.지금은 컬러 이미지로!)
    
    # 객체를 화면에 띄움
    cv2.imshow('image', img)
    
    cv2.waitKey(0) # 사용자가 입력을 할 때까지 무한히 대기..! = 0
    cv2.destroyAllWindows()
    cv2.waitKey(1) # 이게 없으면 프로그램이 무한루프에 빠짐..! 주피터의 오류임
    
if __name__ == '__main__':
    handle_image()

In [8]:
# OpenCV - 이미지 읽기, 쓰기 및 표시하기 (2)

import cv2

def handle_image():
    imgfile = 'images/sample.png'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE) # IMREAD_GRAYSCALE : 흑백 이미지로 불러옴
    
    cv2.namedWindow('image', cv2.WINDOW_NORMAL) # Window의 이름을 설정. 아까 띄운 창은 크기를 조절할 수 없었는데. 지금은 조절 가능.
    # window_normal은 기본 크기로 불러오되, 사용자가 크기 조절 가능
    cv2.imshow('image', img)
    k = cv2.waitKey(0)
    # wait for ESC key to exit
    if k == 27:
        # 키보드 입력을 한 것이 아스키 코드 27라면(esc) 그냥 창을 종료한다
        cv2.destroyAllWindows()
        cv2.waitKey(1)
    # wait for 's' key to save and exit
    elif k == ord('s'):
        # s키를 누르면 grayImage.png로 저장한다
        cv2.imwrite('grayImage.png', img) # 저장
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
if __name__ == '__main__':
    handle_image()

In [23]:
# OpenCV - 도형 외곽 추출하기 (1)

import cv2


def contour():
    imgfile = 'images/sample3.jpg'
    img = cv2.imread(imgfile)
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # cvtColor : 불러온 이미지의 색상 영역을 바꾸겠다. BGR2GRAY : RGB에서 GRAY로 바꿈
    # 외곽을 찾을 때 칼라 이미지가 있으면 불필요한 연산이 있음!
    # 애초에 그레이로 불러오면 되지 않나? => 원래 객체(칼라본) 위에 외곽을 두기 위해 이런 것! 그레이는 외곽용
    
    
    edge = cv2.Canny(imgray, 100, 200)
    # cv2.Canny : 가장 인기있는 edge 알고리즘 ( 이미지 객체, 해당 값보다 작으면 엣지가 아님, 해당 값보다 높으면 엣지 )
    
    # 이해가 안되니 설명하자면 모든 Edge를 검출한 후 맥시멈 Value와 연결되어 있으면 엣지로 봄!
    
    images, contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    # findContours : Contour는 외곽임 () -> 이진화되있는 흰색/검은색만 가지고 나와야 함 ( 외곽, 외각을 찾는 방식. retr_tree, chain_approw_simple)
    
    cv2.imshow('edge', edge)
    cv2.drawContours(img, contours, -1, (0, 255, 0), 1) # 이미지 객체, 컨트어즈 객체, 컨투어 인덱스 파라미터(-1은 전체. 0은 0번째 객체), 외곽 그릴 선의 BGR, 선의 두께
    
    
    cv2.imshow('Contour', img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    contour() 

In [24]:
# OpenCV - 도형 외곽 추출하기 (2) [ 문서가 조금 구겨져 있거나 정확한 직사각형이 아닌 경우!  ]

import cv2

def contour_approx():
    imgfile = 'images/sample6.png'
    img = cv2.imread(imgfile)
    img2 = img.copy()
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    
    edge = cv2.Canny(imgray, 100, 200) 
    
    images, contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cnt = contours[0] # 첫번째 인덱스만 불러옴
    cv2.drawContours(img, [cnt], 0, (0, 255, 0), 3)
    
    epsilon = 0.1 * cv2.arcLength(cnt, True) # arcLength : 검출한 외곽의 둘러의 길이를 구함. 이것의 10%를 입실론이라고 칭함(=>오차라고 보면 됨)
    
    approx = cv2.approxPolyDP(cnt, epsilon, True) # 입실론이 작으면 원래 컨투어와 비슷하고 클수록 꼭지점의 개수가 줄어듬!
    
    cv2.drawContours(img2, [approx], 0, (0, 255, 0), 3)
    
    cv2.imshow('Contour', img) # 그냥 읽은 것
    cv2.imshow('Approx', img2) # 엡실론 적용
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    contour_approx() 

In [12]:
# OpenCV - 투영변환 구현하기 (1)

import numpy as np
import cv2

def warp_affine():
    img = cv2.imread('images/sample4.png')
    
    pts1 = np.float32([[50, 50], [200, 50], [20, 200]]) # 50,50을
    pts2 = np.float32([[70, 100], [220, 50], [150, 250]]) # 70, 100으로 바꾸겠다 => 주변에 있는 것들도 따라서 팽창
    # 포인트를 지정하지 않고 자동으로 하려면 아래 코드 ㄱㄱ
    
    M = cv2.getAffineTransform(pts1, pts2) # 변환!
    
    result = cv2.warpAffine(img, M, (350, 300)) # 이미지 객체와 넣고 출력할 이미지의 크기 정보 지정
    
    cv2.imshow('original', img)
    cv2.imshow('Affine Transform', result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    warp_affine()

In [25]:
# OpenCV - 투영변환 구현하기 (2)

import numpy as np
import cv2

def warp_perspective():
    img = cv2.imread('images/sample5.jpg')
    
    topLeft = [127, 157]
    topRight = [448, 152]
    bottomRight = [579, 526]
    bottomLeft = [54, 549]
    
    # 외곽을 통해 4개의 좌표가 있는 경우! 사각형이 생김
    # 너비와 높이가 도출되는데, 최대값을 추출하는 법과 최소값을 추출. 이번엔 최소값을 구해볼 것임
    
    pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft]) # 꼭지점 설정
    
    w1 = abs(bottomRight[0] - bottomLeft[0]) # 절대값을 구함. 너비1
    w2 = abs(topRight[0] - topLeft[0]) # 너비2
    h1 = abs(topRight[1] - bottomRight[1]) # 높이1
    h2 = abs(topLeft[1] - bottomLeft[1]) # 높이2
    minWidth = min([w1, w2]) # 최소 너비
    minHeight = min([h1, h2]) # 최소 높이
    
    pts2 = np.float32([[0,0], [minWidth-1,0],  #-1을 해서 여백 제거
                      [minWidth-1,minHeight-1], [0,minHeight-1]])
    
    M = cv2.getPerspectiveTransform(pts1, pts2) # 위엔 getaffain을 사용!
    # affain : 좌표 그대로 가져와서 늘리거나 줄임. 원근은 상관 없음!
    # perspective : 원근 고정법. 원근까지 조절함!
    
    result = cv2.warpPerspective(img, M, (int(minWidth), int(minHeight)))
    
    
    cv2.imshow('original', img)
    cv2.imshow('Warp Transform', result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    warp_perspective()

In [26]:
# OpenCV - 스캔한 듯한 효과 주기 (1)
# 스캔한 효과 : 조명의 영향을 제거하기 위해서!

import numpy as np
import cv2

# Callback Function for Trackbar (but do not any work)
def nothing(x):
    pass

def global_threshold():
    # 경계값을 넘으면 검정, 아니면 흰색
    imgfile = 'images/document.jpg'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    # Resize image
    # 이미지 사이즈를 고려하지 않으면 과부하가 걸리는 경우가 많음
    r = 600.0 / img.shape[0] # 가로 픽셀을 600으로 고정 => r은 비율
    dim = (int(img.shape[1] * r), 600)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    
    
    WindowName = "Window"
    TrackbarName = "Threshold"
    
    # Make Window and Trackbar
    cv2.namedWindow(WindowName)
    cv2.createTrackbar(TrackbarName, WindowName, 50, 255, nothing) # 지정한 Trackbar를 바꾸면 어떤 효과가 나오나 볼 것임
    # (트랙바이름, 윈도우이름, 초기, 최대값, 콜백함수)
    
    # Allocate destination image
    Threshold = np.zeros(img.shape, np.uint8)
    # 이진화할 객체 생성. np.uint8?
    
    # Loop for get trackbar pos and process it
    while True:
        # Get position in trackbar
        TrackbarPos = cv2.getTrackbarPos(TrackbarName, WindowName) # 트랙바의 위치를 반환
        # Apply threshold
        cv2.threshold(img, TrackbarPos, 255, cv2.THRESH_BINARY, Threshold) # 그 값이 Threshold임
        # Show in window
        cv2.imshow(WindowName, Threshold)
        
        # 직접 Threshold를 설정할 수 없음..! 이걸 보정하는 방법을 찾아보자. => 이미지를 쪼개서 영역별로 따로 Threshold르 ㄹ구해보자
        
        # wait for ESC key to exit
        k = cv2.waitKey(0)
        if k == 27:
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
    return

if __name__ == '__main__':
    global_threshold() 

In [27]:
# OpenCV - 스캔한 듯한 효과 주기 (2)

import numpy as np
import cv2

def adaptive_threshold():
    imgfile = 'images/document.jpg'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    # Resize image
    r = 600.0 / img.shape[0]
    dim = (int(img.shape[1] * r), 600)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    # Blur image and apply adaptive threshold
    blur = cv2.GaussianBlur(img, (5, 5), 0) # blur 효과를 줌. 가우시안블러 ( 노이즈 제거 -> 약간 흐릿하게 만드는 효과  => 이미지 이진화할 때 뭉개주면 더 결과 좋음 )
    result_without_blur = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)
    # 이미지를 잘게 쪼개서 각자의 Thresold를 구하는 함수. 21:이미지를 얼마나 잘게 쪼갤까. 작으면 작게 쪼개는 것, 10: 주변 영역 밝기의 평균에서 뺄 값! (일반적으로 10)
    result_with_blur = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)
    cv2.imshow('Without Blur', result_without_blur)
    cv2.imshow('With Blur', result_with_blur)
    
    # 블러를 주면 글자를 뭉갬. 글자가 작으면 블러효과를 주지 않는 것이 더 좋음!
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    adaptive_threshold() 

In [33]:
# 명함인식 구현하기

import numpy as np
import cv2

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

def auto_scan_image():
    # load the image and compute the ratio of the old height
    # to the new height, clone it, and resize it
    # document.jpg ~ docuemnt7.jpg
    image = cv2.imread('images/document.jpg')
    orig = image.copy() # 카피함
    r = 800.0 / image.shape[0]
    dim = (int(image.shape[1] * r), 800)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

    # convert the image to grayscale, and find edges
    # in the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 75, 200)

    # show the original image and the edge detected image
    print("STEP 1: Edge Detection")
    cv2.imshow("Image", image)
    cv2.imshow("Edged", edged)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    # find the contours in the edged image, keeping only the
    # largest ones, and initialize the screen contour
    # 외곽을 구해봅시다
    (images, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours 
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # 근사를 했음

        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            # 근사의 꼭지점이 4개일 때 그게 외곽!
            screenCnt = approx
            break

    # show the contour (outline) of the piece of paper
    print("STEP 2: Find contours of paper")
    cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
    cv2.imshow("Outline", image)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    # apply the four point transform to obtain a top-down
    # view of the original image
    # 반듯한 사각형으로 만들기
    rect = order_points(screenCnt.reshape(4, 2) / r) # order_points: 외곽에서 점을 순서대로 추출해 하나의 배열을 반환!
    (topLeft, topRight, bottomRight, bottomLeft) = rect
    
    w1 = abs(bottomRight[0] - bottomLeft[0])
    w2 = abs(topRight[0] - topLeft[0])
    h1 = abs(topRight[1] - bottomRight[1])
    h2 = abs(topLeft[1] - bottomLeft[1])
    maxWidth = max([w1, w2])
    maxHeight = max([h1, h2])
    
    dst = np.float32([[0,0], [maxWidth-1,0], 
                      [maxWidth-1,maxHeight-1], [0,maxHeight-1]])
    
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(orig, M, (maxWidth, maxHeight)) # 이번엔 최대값을 사용

    # show the original and scanned images
    # 스캔!
    
    print("STEP 3: Apply perspective transform")
    cv2.imshow("Warped", warped)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    # convert the warped image to grayscale, then threshold it
    # to give it that 'black and white' paper effect
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY) # 회색으로 바꿔서
    warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10) # 사용하면 스캔한 효과가 나옴(이진화)

    # show the original and scanned images
    print("STEP 4: Apply Adaptive Threshold")
    cv2.imshow("Original", orig)
    cv2.imshow("Scanned", warped)
    cv2.imwrite('scannedImage.png', warped)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    auto_scan_image()

STEP 1: Edge Detection
STEP 2: Find contours of paper
STEP 3: Apply perspective transform
STEP 4: Apply Adaptive Threshold


In [37]:
# OCR - Tesseract
# 글자인식 ( OCR ) -> Train된 서체..!


from PIL import Image
import pytesseract
# http://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-setup-3.05.00dev.exe

def ocr_tesseract():
    image_file = 'scannedImage.png'
    im = Image.open(image_file)
    # 윈도우의 경우 주석 해제
    pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files (x86)/Tesseract-OCR/tesseract'
    text = pytesseract.image_to_string(im) # 불러온 이미지를 저 함수에 넣으면 바로 반환됨! ( 파이썬 3은 유니코드 디코드 에러 뜸)
    im.show()

    print(text)

if __name__ == '__main__':
    ocr_tesseract()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 3: illegal multibyte sequence

In [ ]:
# OCR - Project Oxford by MS
# 조금 더 성능이 좋은 Project Oxford => 마이크로소프트에서 만듬. projectoxford.ai 에서 가입 후..! free key 받기 


from PIL import Image
import httplib, urllib, base64, json

def print_text(json_data):
    result = json.loads(json_data)
    for l in result['regions']:
        for w in l['lines']:
            line = []
            for r in w['words']:
                line.append(r['text'])
            print ' '.join(line)
    return

def ocr_project_oxford(headers, params, data):
    conn = httplib.HTTPSConnection('api.projectoxford.ai')
    conn.request("POST", "/vision/v1.0/ocr?%s" % params, data, headers=headers) # api request 주소 작성
    response = conn.getresponse()
    data = response.read()
    print data + "\n"
    print_text(data)
    conn.close()
    return
    
if __name__ == '__main__':
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': '', # project oxford 인증키 입력
    }
    params = urllib.urlencode({
        # Request parameters
        'language': 'unk', # unknown 알아서 인식해라!
        'detectOrientation ': 'true', # 기울어진 정도까지 detect할 것이냐
    })
    data = open('scannedImage.png', 'rb').read() # 영역도 추출해줌
    
    try:
        image_file = 'scannedImage.png'
        im = Image.open(image_file)
        im.show()
        ocr_project_oxford(headers, params, data)
    except Exception as e:
        print(e)

In [ ]:
# OpenCV - 얼굴인식 구현하기 (1)

import numpy as np
import cv2

def face_detection():
    face_xml = 'haarcascades/haarcascade_frontalface_default.xml'
    eye_xml = 'haarcascades/haarcascade_eye.xml'
    face_cascade = cv2.CascadeClassifier(face_xml)
    eye_cascade = cv2.CascadeClassifier(eye_xml)
    
    img = cv2.imread('images/people.jpg')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), (0,255,0), 2)
    
    cv2.imshow('result', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
if __name__ == '__main__':
    face_detection()# (참고) OpenCV - 이미지에서 텍스트 영역만 찾아내기

# 출처: http://www.danvk.org/2015/01/07/finding-blocks-of-text-in-an-image-using-python-opencv-and-numpy.html
# ocr을 입히기 전에 글자 영역을 찾아내자!
# 바깥은 제외
# 팽창해서 덩어리
# 덩어리를 component

import glob
import os
import random
import sys
import random
import math
import json
from collections import defaultdict

import cv2
from PIL import Image, ImageDraw
import numpy as np
from scipy.ndimage.filters import rank_filter


def dilate(ary, N, iterations): 
    """Dilate using an NxN '+' sign shape. ary is np.uint8."""
    kernel = np.zeros((N,N), dtype=np.uint8)
    kernel[(N-1)/2,:] = 1
    dilated_image = cv2.dilate(ary / 255, kernel, iterations=iterations)

    kernel = np.zeros((N,N), dtype=np.uint8)
    kernel[:,(N-1)/2] = 1
    dilated_image = cv2.dilate(dilated_image, kernel, iterations=iterations)
    return dilated_image


def props_for_contours(contours, ary):
    """Calculate bounding box & the number of set pixels for each contour."""
    c_info = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        c_im = np.zeros(ary.shape)
        cv2.drawContours(c_im, [c], 0, 255, -1)
        c_info.append({
            'x1': x,
            'y1': y,
            'x2': x + w - 1,
            'y2': y + h - 1,
            'sum': np.sum(ary * (c_im > 0))/255
        })
    return c_info


def union_crops(crop1, crop2):
    """Union two (x1, y1, x2, y2) rects."""
    x11, y11, x21, y21 = crop1
    x12, y12, x22, y22 = crop2
    return min(x11, x12), min(y11, y12), max(x21, x22), max(y21, y22)


def intersect_crops(crop1, crop2):
    x11, y11, x21, y21 = crop1
    x12, y12, x22, y22 = crop2
    return max(x11, x12), max(y11, y12), min(x21, x22), min(y21, y22)


def crop_area(crop):
    x1, y1, x2, y2 = crop
    return max(0, x2 - x1) * max(0, y2 - y1)


def find_border_components(contours, ary):
    borders = []
    area = ary.shape[0] * ary.shape[1]
    for i, c in enumerate(contours):
        x,y,w,h = cv2.boundingRect(c)
        if w * h > 0.5 * area:
            borders.append((i, x, y, x + w - 1, y + h - 1))
    return borders


def angle_from_right(deg):
    return min(deg % 90, 90 - (deg % 90))


def remove_border(contour, ary):
    """Remove everything outside a border contour."""
    # Use a rotated rectangle (should be a good approximation of a border).
    # If it's far from a right angle, it's probably two sides of a border and
    # we should use the bounding box instead.
    c_im = np.zeros(ary.shape)
    r = cv2.minAreaRect(contour)
    degs = r[2]
    if angle_from_right(degs) <= 10.0:
        box = cv2.cv.BoxPoints(r)
        box = np.int0(box)
        cv2.drawContours(c_im, [box], 0, 255, -1)
        cv2.drawContours(c_im, [box], 0, 0, 4)
    else:
        x1, y1, x2, y2 = cv2.boundingRect(contour)
        cv2.rectangle(c_im, (x1, y1), (x2, y2), 255, -1)
        cv2.rectangle(c_im, (x1, y1), (x2, y2), 0, 4)

    return np.minimum(c_im, ary)


def find_components(edges, max_components=16):
    """Dilate the image until there are just a few connected components.
    Returns contours for these components."""
    # Perform increasingly aggressive dilation until there are just a few
    # connected components.
    count = 21
    dilation = 5
    n = 1
    while count > 16:
        n += 1
        dilated_image = dilate(edges, N=3, iterations=n)
        contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        count = len(contours)
    #print dilation
    #Image.fromarray(edges).show()
    #Image.fromarray(255 * dilated_image).show()
    return contours


def find_optimal_components_subset(contours, edges):
    """Find a crop which strikes a good balance of coverage/compactness.
    Returns an (x1, y1, x2, y2) tuple.
    """
    c_info = props_for_contours(contours, edges)
    c_info.sort(key=lambda x: -x['sum'])
    total = np.sum(edges) / 255
    area = edges.shape[0] * edges.shape[1]

    c = c_info[0]
    del c_info[0]
    this_crop = c['x1'], c['y1'], c['x2'], c['y2']
    crop = this_crop
    covered_sum = c['sum']

    while covered_sum < total:
        changed = False
        recall = 1.0 * covered_sum / total
        prec = 1 - 1.0 * crop_area(crop) / area
        f1 = 2 * (prec * recall / (prec + recall))
        #print '----'
        for i, c in enumerate(c_info):
            this_crop = c['x1'], c['y1'], c['x2'], c['y2']
            new_crop = union_crops(crop, this_crop)
            new_sum = covered_sum + c['sum']
            new_recall = 1.0 * new_sum / total
            new_prec = 1 - 1.0 * crop_area(new_crop) / area
            new_f1 = 2 * new_prec * new_recall / (new_prec + new_recall)

            # Add this crop if it improves f1 score,
            # _or_ it adds 25% of the remaining pixels for <15% crop expansion.
            # ^^^ very ad-hoc! make this smoother
            remaining_frac = c['sum'] / (total - covered_sum)
            new_area_frac = 1.0 * crop_area(new_crop) / crop_area(crop) - 1
            if new_f1 > f1 or (
                    remaining_frac > 0.25 and new_area_frac < 0.15):
                print '%d %s -> %s / %s (%s), %s -> %s / %s (%s), %s -> %s' % (
                        i, covered_sum, new_sum, total, remaining_frac,
                        crop_area(crop), crop_area(new_crop), area, new_area_frac,
                        f1, new_f1)
                crop = new_crop
                covered_sum = new_sum
                del c_info[i]
                changed = True
                break

        if not changed:
            break

    return crop


def pad_crop(crop, contours, edges, border_contour, pad_px=15):
    """Slightly expand the crop to get full contours.
    This will expand to include any contours it currently intersects, but will
    not expand past a border.
    """
    bx1, by1, bx2, by2 = 0, 0, edges.shape[0], edges.shape[1]
    if border_contour is not None and len(border_contour) > 0:
        c = props_for_contours([border_contour], edges)[0]
        bx1, by1, bx2, by2 = c['x1'] + 5, c['y1'] + 5, c['x2'] - 5, c['y2'] - 5

    def crop_in_border(crop):
        x1, y1, x2, y2 = crop
        x1 = max(x1 - pad_px, bx1)
        y1 = max(y1 - pad_px, by1)
        x2 = min(x2 + pad_px, bx2)
        y2 = min(y2 + pad_px, by2)
        return crop
    
    crop = crop_in_border(crop)

    c_info = props_for_contours(contours, edges)
    changed = False
    for c in c_info:
        this_crop = c['x1'], c['y1'], c['x2'], c['y2']
        this_area = crop_area(this_crop)
        int_area = crop_area(intersect_crops(crop, this_crop))
        new_crop = crop_in_border(union_crops(crop, this_crop))
        if 0 < int_area < this_area and crop != new_crop:
            print '%s -> %s' % (str(crop), str(new_crop))
            changed = True
            crop = new_crop

    if changed:
        return pad_crop(crop, contours, edges, border_contour, pad_px)
    else:
        return crop


def downscale_image(im, max_dim=2048):
    """Shrink im until its longest dimension is <= max_dim.
    Returns new_image, scale (where scale <= 1).
    """
    a, b = im.size
    if max(a, b) <= max_dim:
        return 1.0, im

    scale = 1.0 * max_dim / max(a, b)
    new_im = im.resize((int(a * scale), int(b * scale)), Image.ANTIALIAS)
    return scale, new_im


def process_image(path, out_path):
    orig_im = Image.open(path)
    scale, im = downscale_image(orig_im)

    edges = cv2.Canny(np.asarray(im), 100, 200)

    # TODO: dilate image _before_ finding a border. This is crazy sensitive!
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    borders = find_border_components(contours, edges)
    borders.sort(key=lambda (i, x1, y1, x2, y2): (x2 - x1) * (y2 - y1))

    border_contour = None
    if len(borders):
        border_contour = contours[borders[0][0]]
        edges = remove_border(border_contour, edges)

    edges = 255 * (edges > 0).astype(np.uint8)

    # Remove ~1px borders using a rank filter.
    maxed_rows = rank_filter(edges, -4, size=(1, 20))
    maxed_cols = rank_filter(edges, -4, size=(20, 1))
    debordered = np.minimum(np.minimum(edges, maxed_rows), maxed_cols)
    edges = debordered

    contours = find_components(edges)
    if len(contours) == 0:
        print '%s -> (no text!)' % path
        return

    crop = find_optimal_components_subset(contours, edges)
    crop = pad_crop(crop, contours, edges, border_contour)

    crop = [int(x / scale) for x in crop]  # upscale to the original image size.
    
    # draw and show cropped rectangle area in the original image
    draw = ImageDraw.Draw(im)
    draw.rectangle(crop, outline='red')
    im.show()
    
    text_im = orig_im.crop(crop)
    text_im.save(out_path)
    print '%s -> %s' % (path, out_path)


if __name__ == '__main__':
    path = 'images/text.jpg'
    out_path = 'cropped_document.png'
    try:
        process_image(path, out_path)
    except Exception as e:
        print '%s %s' % (path, e)